# 1. Tensorflow 2.0 기본

## 1.1 Eager execution(즉시 실행)

텐서플로의 즉시 실행은 그래프를 생성하지 않고 함수를 바로 실행하는 명령형 프로그래밍 환경.
나중에 실행하기 위해 계산가능한 그래프를 생성하는 대신에 계산값을 즉시 알려주는 연산
이러한 기능은 

In [1]:
import tensorflow as tf

In [2]:
print(tf.__version__)

2.1.0


In [3]:
x = [[1,2]]
y = [[3],[4]]
m = tf.matmul(x,y)
m

<tf.Tensor: shape=(1, 1), dtype=int32, numpy=array([[11]], dtype=int32)>

In [4]:
a = tf.constant([[1,2],[3,4]]) #상수는 내부값을 못바꾼다
print(a)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)


In [5]:
# 브로드캐스팅 지원
b = tf.add(a, 1)
print(b)

tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)


In [6]:
# 연산자 오버로딩 지원
print(a*b)

tf.Tensor(
[[ 2  6]
 [12 20]], shape=(2, 2), dtype=int32)


In [7]:
import numpy as np

In [8]:
c = np.multiply(a, b)
print(c)

[[ 2  6]
 [12 20]]


## 그래디언트 계산하기
자동 미분은 인공신경망 훈련을 위한 역전파와 같은 기계학습 알고리즘을 구현하는데 유용. 즉시 실행을 사용하는 동안에는, 나중에 그래디언트를 계산하는 연산을 추적하기 위해 tf.GradientTape을 사용
즉시 실행중에 그래디언트를 계산하고 모델 훈련에 이용하기 위해서 tf.GradientTape 사용할 수 있다. 특히 복잡하고 반복적인 훈련인 경우에 더 유용
매번 실행될 때 서로 다른 연산이 수행될 수 있기 때문에 모든 정방향 연산은 tape에 기록된다. 그 다음 tape을 거꾸로 돌려 그래디언트를 계산후 tape을 폐기한다. 특정한 tf.GradientTape은 오직 하나의 그래디언트만을 계산할 수 있고 부가적인 호출은 실행 중 에러를 발생

In [10]:
w = tf.Variable([[1.0]])
with tf.GradientTape() as tape:
    loss = w * w
    
grad = tape.gradient(loss, w)
print(grad)

tf.Tensor([[2.]], shape=(1, 1), dtype=float32)


## 모델 훈련
다음 예는 표준 MNIST 손글씨 분류를 위한 다층 모델을 생성, 즉시 실행 환경에서 훈련 가능한 그래프를 생성하기 위한 옵티마이저와 층 API를 보여준다.

In [11]:
# mnist 데이터 가져오기 및 포맷 맞추기
(mnist_images, mnist_labels), _ = tf.keras.datasets.mnist.load_data()

dataset = tf.data.Dataset.from_tensor_slices((tf.cast(mnist_images[...,tf.newaxis]/255, tf.float32), tf.cast(mnist_labels, tf.int64)))
dataset = dataset.shuffle(1000).batch(32)

In [12]:
# 모델 생성
mnist_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,[3,3],activation='relu',
                          input_shape=(None,None,1)),
    tf.keras.layers.Conv2D(16,[3,3],activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(10)
])

즉시 실행에서는 훈련을 하지 않아도 모델을 사용하고 결과를 점검할 수 있다.

In [13]:
for images, labels in dataset.take(1):
    print('로짓: ', mnist_model(images[0:1]).numpy())

로짓:  [[-0.02189998 -0.02657841 -0.00553481 -0.02905848 -0.01789952  0.00336728
   0.03077359 -0.04671326 -0.0267408  -0.03950467]]


케라스 모델은 자체적인 훈련 메서드(fit)을 포함하고 있지만 때로는 좀 더 수정할 필요가 있다. 다음은 즉시 실행을 활용한 반복적인 훈련의 예이다.

## 변수와 옵티마이저

In [15]:
class Model(tf.keras.Model):
    def __init__(self):
        super(Model, self).__init__()
        self.W = tf.Variable(5., name='weight')
        self.B = tf.Variable(10., name='bias')
    def call(self, inputs):
        return inputs * self.W + self.B
    
# 약 3 * 2 + 2개의 점으로 구성된 실험데이터
NUM_EXAMPLES = 2000
training_inputs = tf.random.normal([NUM_EXAMPLES])
noise = tf.random.normal([NUM_EXAMPLES])
training_outputs = training_inputs * 3 + 2 + noise

# 최적화 손실함수
def loss(model, inputs, targets):
    error = model(inputs) - targets
    return tf.reduce_mean(tf.square(error))

def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
    return tape.gradient(loss_value, [model.W, model.B])

# 정의:
# 1. 모델
# 2. 모델 파라미터에 대한 손실 함수의 미분
# 3. 미분에 기초한변수 업데이트 전략

model = Model()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

print("초기 손실:{:.3f}".format(loss(model, training_inputs, training_outputs)))

# 반복훈련
for i in range(1000):
    grads = grad(model, training_inputs, training_outputs)
    optimizer.apply_gradients(zip(grads, [model.W, model.B]))
    if i % 20 == 0:
        print("스텝 {:03d}에서 손실: {:.3f}".format(i, loss(model, training_inputs, training_outputs)))

print("최종 손실: {:.3f}".format(loss(model, training_inputs, training_outputs)))
print("W = {}, B = {}".format(model.W.numpy(), model.B.numpy()))

초기 손실:69.980
스텝 000에서 손실: 67.226
스텝 020에서 손실: 30.320
스텝 040에서 손실: 13.987
스텝 060에서 손실: 6.758
스텝 080에서 손실: 3.559
스텝 100에서 손실: 2.143
스텝 120에서 손실: 1.516
스텝 140에서 손실: 1.238
스텝 160에서 손실: 1.115
스텝 180에서 손실: 1.061
스텝 200에서 손실: 1.037
스텝 220에서 손실: 1.026
스텝 240에서 손실: 1.021
스텝 260에서 손실: 1.019
스텝 280에서 손실: 1.018
스텝 300에서 손실: 1.018
스텝 320에서 손실: 1.018
스텝 340에서 손실: 1.018
스텝 360에서 손실: 1.018
스텝 380에서 손실: 1.018
스텝 400에서 손실: 1.018
스텝 420에서 손실: 1.018
스텝 440에서 손실: 1.018
스텝 460에서 손실: 1.018
스텝 480에서 손실: 1.018
스텝 500에서 손실: 1.018
스텝 520에서 손실: 1.018
스텝 540에서 손실: 1.018
스텝 560에서 손실: 1.018
스텝 580에서 손실: 1.018
스텝 600에서 손실: 1.018
스텝 620에서 손실: 1.018
스텝 640에서 손실: 1.018
스텝 660에서 손실: 1.018
스텝 680에서 손실: 1.018
스텝 700에서 손실: 1.018
스텝 720에서 손실: 1.018
스텝 740에서 손실: 1.018
스텝 760에서 손실: 1.018
스텝 780에서 손실: 1.018
스텝 800에서 손실: 1.018
스텝 820에서 손실: 1.018
스텝 840에서 손실: 1.018
스텝 860에서 손실: 1.018
스텝 880에서 손실: 1.018
스텝 900에서 손실: 1.018
스텝 920에서 손실: 1.018
스텝 940에서 손실: 1.018
스텝 960에서 손실: 1.018
스텝 980에서 손실: 1.018
최종 손실: 1.018
W = 3.01543927192688,

변수는 객체, 즉시 실행에서는 변수는 그 객체의 마지막 참조가 제거될 때까지 유지되고 그 이후 삭제됩니다.

In [ ]:
v = tf.Variable()

## 1.2 tensor 기초

스칼라 기본

In [17]:
rank_0_tensor = tf.constant(4)
print(rank_0_tensor)

tf.Tensor(4, shape=(), dtype=int32)


1차원 벡터

In [18]:
rank_1_tensor = tf.constant([2.0, 3.0, 4.0])
print(rank_1_tensor)

tf.Tensor([2. 3. 4.], shape=(3,), dtype=float32)


2차원 행렬, 랭크2

In [19]:
rank_2_tensor = tf.constant([[1,2],[3,4],[5,6]])
print(rank_2_tensor)

tf.Tensor(
[[1 2]
 [3 4]
 [5 6]], shape=(3, 2), dtype=int32)


3차원 텐서

In [20]:
rank_3_tensor = tf.constant([[[1,2,3,4],[5,6,7,8]],[[9,10,11,12],[13,14,15,16]],[[17,18,19,20],[21,22,23,24]]])
print(rank_3_tensor)

tf.Tensor(
[[[ 1  2  3  4]
  [ 5  6  7  8]]

 [[ 9 10 11 12]
  [13 14 15 16]]

 [[17 18 19 20]
  [21 22 23 24]]], shape=(3, 2, 4), dtype=int32)


In [23]:
a = tf.constant([[4.,5.],[10.,1.]])

print(tf.reduce_max(a))
print(tf.argmax(a))
print(tf.nn.softmax(a))

tf.Tensor(10.0, shape=(), dtype=float32)
tf.Tensor([1 0], shape=(2,), dtype=int64)
tf.Tensor(
[[2.6894143e-01 7.3105860e-01]
 [9.9987662e-01 1.2339458e-04]], shape=(2, 2), dtype=float32)
